In [1]:
#add parent directory to sys.path to allow module imports
#see https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder/11158224#11158224
import sys
sys.path.insert(1,'..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
from helpers import fileload as fl
from helpers.calc_chemfeat import perovskite, formula_redfeat, formula_pif
from helpers.predict import predict_from_pifs
import pypif.pif
from pypif.obj import *
import copy
from citrination_client import CitrinationClient
import pickle

['..',
 '',
 'C:\\Users\\jdh10\\Anaconda3\\python36.zip',
 'C:\\Users\\jdh10\\Anaconda3\\DLLs',
 'C:\\Users\\jdh10\\Anaconda3\\lib',
 'C:\\Users\\jdh10\\Anaconda3',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\PyDispatcher-2.0.5-py3.6.egg',
 'C:\\Users\\jdh10\\OneDrive - Colorado School of Mines\\python\\misc_modules',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\monty-1.0.2-py3.6.egg',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\palettable-3.1.0-py3.6.egg',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\tabulate-0.8.2-py3.6.egg',
 'C:\\Users\\jdh10\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\jdh10\\.ipython']

In [17]:
tcodir = os.path.join(os.environ['HOME'],'OneDrive - Colorado School of Mines/Research/MIDDMI/TCO')
datadir = os.path.join(tcodir,'data/conductivity') #data/EIS_data
pifdir = os.path.join(datadir,'pifs')
pickledir = os.path.join(tcodir,'scripts/pickles')

def sampledir(sample,src):
    return os.path.join(datadir,src,sample)

#load pickles
calc_feat_Co23Fe34 = pickle_dict(os.path.join(pickledir,'feat_Co23Fe34'))
calc_feat_Co24Fe24 = pickle_dict(os.path.join(pickledir,'feat_Co24Fe24'))
calc_feat_Co34Fe23 = pickle_dict(os.path.join(pickledir,'feat_Co34Fe23'))

In [31]:
cat_ox_lims = {'Co':[2,4],'Fe':[2,4]}
dest = 'Co(2,4)Fe(2,4)'
src = 'in'

destdir = os.path.join(pifdir,dest)

samples = next(os.walk(os.path.join(datadir,src)))[1]

# #save red_feat for reuse
# try:
#     calc_feat_Co24Fe24 #{(sample,point):red_feat}
# except NameError:
#     calc_feat_Co24Fe24 = {}

# try:
#     calc_feat_Co23Fe34
# except NameError:
#     calc_feat_Co23Fe34 = {}
    
# try: #for testing physicality of model
#     calc_feat_Co34Fe23
# except NameError:
#     calc_feat_Co34Fe23 = {}
    
for sample in samples:
    print('------------------\nProcessing {}\n------------------'.format(sample))
    df = fl.load_sample_files(sampledir(sample,src),info_filter={'T_set':['500C'],'atm':'dry'})
    fl.get_formula(df,overwrite=True)
    df = df.loc[df['Sigma_elec']>0,:]
    
    df['log_sigma'] = np.log10(df['Sigma_elec'])

    pifs = {}
    n_feat_calc = 0
    for idx in df.index:
        row = df.loc[idx,:]
        formula = row['formula']
        point = row['Point']
        
        if cat_ox_lims == {'Co':[2,4],'Fe':[2,4]}:
            try:
                red_feat = calc_feat_Co24Fe24[(sample,point)]
            except KeyError:
                red_feat = formula_redfeat(formula,cat_ox_lims=cat_ox_lims)
                calc_feat_Co24Fe24[(sample,point)] = red_feat
                n_feat_calc += 1
        elif cat_ox_lims == {'Co':[2,3],'Fe':[3,4]}:
            try:
                red_feat = calc_feat_Co23Fe34[(sample,point)]
            except KeyError:
                red_feat = formula_redfeat(formula,cat_ox_lims=cat_ox_lims)
                calc_feat_Co23Fe34[(sample,point)] = red_feat
                n_feat_calc += 1
        elif cat_ox_lims == {'Co':[3,4],'Fe':[2,3]}:
            try:
                red_feat = calc_feat_Co34Fe23[(sample,point)]
            except KeyError:
                red_feat = formula_redfeat(formula,cat_ox_lims=cat_ox_lims)
                calc_feat_Co34Fe23[(sample,point)] = red_feat
                n_feat_calc += 1
        else:
            raise Exception('cat_ox_lims {} does not match expected cases'.format(cat_ox_lims))
        
        row_pif, red_feat = formula_pif(formula,cat_ox_lims=cat_ox_lims,red_feat=red_feat)

        "Identifiers"
        sample_num = Id(name='Sample', value=sample[-5:])
        pointid = Id(name='Point',value=int(point)) 
        row_pif.ids = [sample_num, pointid]

        "Properties"
        #output properties - "labels"
        sigma = Property(name='sigma_e',units='S/cm',scalars=row['Sigma_elec'])
        sigma.conditions = [Value(name='Temperature ($^\circ$C)',scalars=int(row['Temp_C'])),
                            Value(name='Atmosphere',scalars=row['atm'])]

        log_sigma = Property(name='log_sigma_e',scalars=row['log_sigma'])
        log_sigma.conditions = [Value(name='Temperature ($^\circ$C)',scalars=int(row['Temp_C'])),
                            Value(name='Atmosphere',scalars=row['atm'])]

        row_pif.properties += [sigma,log_sigma]

        #input propertes - "features"
#             pvskt = perovskite(formula=row['formula'], site_ox_lim={'A':[2,4],'B':[2,4]},site_base_ox={'A':2,'B':4})
#             #pvskt.set_cat_ox_lim('Co',[2,3])
#             pvskt.featurize()
#             props_oxavg = props.copy()
#             #filtered_feat = {k:v for (k,v) in pvskt.features.items() if k[-5:] not in ['oxmin','oxmax']}
#             for feat, val in pvskt.features.items():
#                 prop = Property(name=feat,scalars=val)
#                 props.append(prop)
#                 if feat[-5:] not in ['oxmin','oxmax'] and feat[0:7]!='O_delta':
#                     props_oxavg.append(prop)
#             row_pif_oxavg = copy.copy(row_pif)
#             row_pif.properties = props
#             row_pif_oxavg.properties = props_oxavg
#             oxavg_pifs.append(row_pif_oxavg)
        pifs[point] = row_pif
    
    print('Calculated red_feat for {} pifs'.format(n_feat_calc))
    
    outfile = os.path.join(destdir,'{}_oxavg_pif.json'.format(sample))
    file = open(outfile,'w')
    pypif.pif.dump(list(pifs.values()),file,indent=0)
    file.close()    
    print('Dumped {} pifs in {}'.format(len(df), os.path.basename(outfile)))

#     oxavg_outfile = os.path.join(pifdir,'{}_oxavg_pif.json'.format(sample))
#     oxavg_file = open(oxavg_outfile,'w')
#     for rp in oxavg_pifs:
#         pif.dump(rp,oxavg_file)
#     oxavg_file.close()
#     print('Dumped {} pifs in {}'.format(len(oxavg_pifs), os.path.basename(oxavg_outfile)))

#update pickle files with any new entries
calc_feat_Co23Fe34.update_file()
calc_feat_Co24Fe24.update_file()
calc_feat_Co34Fe23.update_file()

------------------
Processing PDAC_COM3_01251
------------------
Calculated red_feat for 0 pifs
Dumped 130 pifs in PDAC_COM3_01251_oxavg_pif.json
------------------
Processing PDAC_COM3_01254
------------------
Calculated red_feat for 0 pifs
Dumped 132 pifs in PDAC_COM3_01254_oxavg_pif.json
------------------
Processing PDAC_COM3_01255
------------------
Calculated red_feat for 0 pifs
Dumped 131 pifs in PDAC_COM3_01255_oxavg_pif.json
------------------
Processing PDAC_COM3_01256
------------------
Calculated red_feat for 0 pifs
Dumped 131 pifs in PDAC_COM3_01256_oxavg_pif.json


In [12]:
"create dataset if not existing"
# client = CitrinationClient(os.environ['CITRINATION_API_KEY'],'https://citrination.com')
# dataset1 = client.create_dataset(name='tco_sigma_elec_full',
#                                 description='Electrical conductivity for TCO thin films with all features',
#                                 public=0)

# dataset2 = client.create_dataset(name='tco_sigma_elec_oxavg_Co[2,3]Fe[3,4]',
#                                 description='Electrical conductivity for TCO thin films with oxavg features only',
#                                 public=0)

# dataset.id, dataset2.id

# dataset4 = client.create_dataset(name='tco_sigma_elec_oxavg_Co[2,4]Fe[2,4]',
#                                 description='Electrical conductivity for TCO thin films with oxavg features only. Cation ox limits: Co: [2,4]; Fe: [2,4]',
#                                 public=0)

dataset5 = client.create_dataset(name='tco_sigma_elec_oxavg_Co[3,4]Fe[2,3]',
                                description='Electrical conductivity for TCO thin films with oxavg features only. Cation ox limits: Co: [3,4]; Fe: [2,3]',
                                public=0)

dataset5.id

162971

### Datasets
tco_sigma_elec_oxavg_**Co[2,3]Fe[3,4]: 162725**

tco_sigma_elec_oxavg_**Co[2,4]Fe[2,4]: 162765**

tco_sigma_elec_oxavg_**Co[3,4]Fe[2,3]: 162971**

In [13]:
"general pif upload"
client = CitrinationClient(os.environ['CITRINATION_API_KEY'],'https://citrination.com')
dsid = 162971 

upload_dir = os.path.join(pifdir,'Co(3,4)Fe(2,3)')

results = []

for file in glob.glob(os.path.join(upload_dir,'*oxavg*.json')):
    result = client.upload(dsid,file)
    results.append(result)

[r.__dict__ for r in results]

[{'_failures': [],
  '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(3,4)Fe(2,3)\\PDAC_COM3_01251_oxavg_pif.json'}]},
 {'_failures': [],
  '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(3,4)Fe(2,3)\\PDAC_COM3_01254_oxavg_pif.json'}]},
 {'_failures': [],
  '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(3,4)Fe(2,3)\\PDAC_COM3_01255_oxavg_pif.json'}]},
 {'_failures': [],
  '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(3,4)Fe(2,3)\\PDAC_COM3_01256_oxavg_pif.json'}]}]

In [139]:
glob.glob(os.path.join('C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(2,4)Fe(2,4)','*'))

['C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(2,4)Fe(2,4)\\PDAC_COM3_01251_oxavg_pif.json',
 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(2,4)Fe(2,4)\\PDAC_COM3_01251_pif.json',
 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(2,4)Fe(2,4)\\PDAC_COM3_01254_oxavg_pif.json',
 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(2,4)Fe(2,4)\\PDAC_COM3_01254_pif.json',
 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(2,4)Fe(2,4)\\PDAC_COM3_01255_oxavg_pif.json',
 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\Co(2,4)Fe(2,4)\\PDAC_COM3_01255_pif.json',
 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pi

In [87]:
"Upload pifs"
client = CitrinationClient(os.environ['CITRINATION_API_KEY'],'https://citrination.com')
dsid1 = 162724 #full
dsid2 = 162725 #oxavg only Co[2,3]Fe[3,4]

results1 = []
results2 = []

for file in glob.glob(os.path.join(pifdir,'*.json')):
    filename = os.path.basename(file)
    if file.find('oxavg') >= 0:
        result = client.upload(dsid2,file)
        results2.append(result)
        #print('oxavg: {}'.format(filename))
    else:
        result = client.upload(dsid1,file)
        results1.append(result)
        #print('full: ' + filename)
        
[r.__dict__ for r in results1], [r.__dict__ for r in results2]

([{'_failures': [],
   '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01251_pif.json'}]},
  {'_failures': [],
   '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01254_pif.json'}]},
  {'_failures': [],
   '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01255_pif.json'}]},
  {'_failures': [],
   '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01256_pif.json'}]}],
 [{'_failures': [],
   '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01251_oxavg_pif.json'}]},
  {'_failures': [],
   '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado Schoo

In [97]:
"dataset excluding 01255"
dataset3 = client.create_dataset(name='tco_sigma_elec_excl01255',
                                description='Electrical conductivity for TCO thin films with oxavg features only and 01255 excluded',
                                public=0)
dataset3.id

162726

In [100]:
"Upload pifs to dataset 3"
dsid3 = 162726 #oxavg only, exclude 01255

results3 = []

for file in glob.glob(os.path.join(pifdir,'*.json')):
    filename = os.path.basename(file)
    if file.find('oxavg') >= 0 and file.find('01255')==-1:
        result = client.upload(dsid3,file)
        results3.append(result)
        print(filename)
        
[r.__dict__ for r in results3]

PDAC_COM3_01251_oxavg_pif.json
PDAC_COM3_01254_oxavg_pif.json
PDAC_COM3_01256_oxavg_pif.json


[{'_failures': [],
  '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01251_oxavg_pif.json'}]},
 {'_failures': [],
  '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01254_oxavg_pif.json'}]},
 {'_failures': [],
  '_successes': [{'path': 'C:\\Users\\jdhuang\\OneDrive - Colorado School of Mines/Research/MIDDMI/TCO\\data/conductivity\\pifs\\PDAC_COM3_01256_oxavg_pif.json'}]}]

In [90]:
data = pd.DataFrame()
samples = next(os.walk(os.path.join(datadir,src)))[1]
for sample in samples:
    df = fl.load_sample_files(sampledir(sample,src),info_filter={'T_set':['500C'],'atm':'dry'})
    df['sample'] = sample
    data = data.append(df,sort=True)
    
neg = data.loc[data['Sigma_elec']<=0,:]
pos = data.loc[data['Sigma_elec']>=0,:]


In [145]:
row_pif.properties[0]

In [96]:
low = data.loc[data['Sigma_elec']<5e-6,:]
for s in low['sample'].unique():
    print(s,len(low[low['sample']==s]))

PDAC_COM3_01251 3
PDAC_COM3_01254 22
PDAC_COM3_01255 131
PDAC_COM3_01256 1


In [101]:
p.features

{'A:B_ratio': 0.96661,
 'A_sum': 0.96661,
 'B_sum': 1.0,
 'Ba_amt': 0.96661,
 'Co:Fe_ratio': 0.8273470764301674,
 'Co_amt': 0.39251,
 'Fe_amt': 0.47442,
 'MO_ABE': -291.9854788027778,
 'MO_IC_a': 0.8032113561415081,
 'MO_IC_avg': 0.6462386718330289,
 'MO_IC_b': 0.49450730545360977,
 'O_delta_oxavg': 0.4668549999999998,
 'O_delta_oxmax': 0.9003199999999998,
 'O_delta_oxmin': 0.03338999999999981,
 'X_a': 0.89,
 'X_avg': 1.3441268985716537,
 'X_b': 1.7830905,
 'X_cat_a_oxavg': 0.7715751707110065,
 'X_cat_a_oxmax': 0.7715751707110065,
 'X_cat_a_oxmin': 0.7715751707110065,
 'X_cat_b_oxavg': 7.924788591730124,
 'X_cat_b_oxmax': 12.161218556124753,
 'X_cat_b_oxmin': 4.000727505644781,
 'Y_amt': 0.0,
 'Zr_amt': 0.13307,
 'acceptor_magnitude': 0.0,
 'alat_oxavg': 4.26342427577157,
 'alat_oxmax': 4.3165559,
 'alat_oxmin': 4.256782822743016,
 'goldschmidt_oxavg': 1.0337057094996953,
 'goldschmidt_oxmax': 1.0741984265945168,
 'goldschmidt_oxmin': 0.9861525997481038,
 'mass_a': 137.327,
 'mass_avg'

In [102]:
import pymatgen as mg

In [117]:
fe = mg.Element('Ba')
fe.average_ionic_radius

1.49

In [118]:
bsite = ['Ba','Co','Fe','Zr','Y']
for x in bsite:
    el = mg.Element(x)
    print(x,el.average_ionic_radius)

Ba 1.49 ang
Co 0.7683333333333334 ang
Fe 0.8525 ang
Zr 0.86 ang
Y 1.04 ang


In [121]:
x = mg.Element('Y')
x.average_ionic_radius, x.data['Shannon radii']

(1.04,
 {'3': {'IX': {'': {'crystal_radius': 1.215, 'ionic_radius': 1.075}},
   'VI': {'': {'crystal_radius': 1.04, 'ionic_radius': 0.9}},
   'VII': {'': {'crystal_radius': 1.1, 'ionic_radius': 0.96}},
   'VIII': {'': {'crystal_radius': 1.159, 'ionic_radius': 1.019}}}})

In [18]:
calc_feat_Co23Fe34

{('PDAC_COM3_01251', 1): {'A:B_ratio': 0.96661,
  'A_sum': 0.96661,
  'B_sum': 1.0,
  'Ba_amt': 0.96661,
  'Co:Fe_ratio': 0.8273470764301674,
  'Co_amt': 0.39251,
  'Fe_amt': 0.47442,
  'MO_ABE': -291.9854788027778,
  'MO_IC_a': 0.8032113561415081,
  'MO_IC_avg': 0.6462386718330289,
  'MO_IC_b': 0.49450730545360977,
  'X_a': 0.89,
  'X_avg': 1.3441268985716537,
  'X_b': 1.7830905,
  'X_cat_a_oxavg': 0.7715751707110065,
  'X_cat_b_oxavg': 7.799352483080915,
  'Y_amt': 0.0,
  'Zr_amt': 0.13307,
  'acceptor_magnitude': 0.0,
  'alat_oxavg': 4.256782822743016,
  'goldschmidt_oxavg': 1.0366904006714908,
  'mass_a': 137.327,
  'mass_avg': 98.904552717341,
  'mass_b': 61.76503094945001,
  'n_a_oxavg': 2.0,
  'n_b_oxavg': 3.1740250000000003,
  'r_a_oxavg': 1.61,
  'r_b_oxavg': 0.653504225,
  'r_b_std_oxavg': 0.050752076888990694,
  'r_crit_oxavg': 0.944320028059585,
  'tau_oxavg': 3.6182201136754575,
  'tot_cat_charge_oxavg': 5.107245000000001,
  'trans_met_amt': 0.86693,
  'uc_vol_free_oxavg':

In [19]:
form_feat_Co23Fe34 = calc_feat_Co23Fe34.copy()

In [20]:
df

,Point,Sigma_elec,T_set,Temp_C,atm,d_um,x_mm,y_mm,formula,log_sigma
Point_info,,,,,,,,,,
1_500C_dry,1,0.000030,500C,437,dry,0.182950,0,0.0,Ba0.85088Co0.42955Fe0.03361Y0.14889Zr0.38795O3,-4.524454
2_500C_dry,2,0.000030,500C,437,dry,0.186953,0,1.5,Ba0.85082Co0.42873Fe0.03342Y0.14908Zr0.38877O3,-4.519989
3_500C_dry,3,0.000030,500C,437,dry,0.190956,0,3.0,Ba0.85075Co0.42791Fe0.03322Y0.14928Zr0.38959O3,-4.526754
4_500C_dry,4,0.000034,500C,437,dry,0.195850,-4,0.0,Ba0.86464Co0.46001Fe0.02896Y0.13812Zr0.37291O3,-4.473622
5_500C_dry,5,0.000034,500C,437,dry,0.200888,-4,1.5,Ba0.8618Co0.45773Fe0.02887Y0.1392Zr0.3742O3,-4.462971
6_500C_dry,6,0.000034,500C,437,dry,0.205925,-4,3.0,Ba0.85895Co0.45545Fe0.02879Y0.14027Zr0.37549O3,-4.470371
7_500C_dry,7,0.000037,500C,437,dry,0.208480,-8,0.0,Ba0.8657Co0.48182Fe0.02622Y0.13163Zr0.36032O3,-4.437294
8_500C_dry,8,0.000038,500C,437,dry,0.214056,-8,1.5,Ba0.86353Co0.48038Fe0.02641Y0.13195Zr0.36125O3,-4.422768
9_500C_dry,9,0.000038,500C,437,dry,0.219633,-8,3.0,Ba0.86136Co0.47895Fe0.02661Y0.13227Zr0.36217O3,-4.420386


In [26]:
from helpers.fileload import get_formula
help(get_formula)

Help on function get_formula in module helpers.fileload:

get_formula(df, overwrite=False)
    derive chemical formula from X_at columns of dataframe
    ----------------
    df: dataframe containing atomic fractions
    overwrite: if True, add formula column to df and remove X_at columns in place. If False, don't modify df, and simply return formulas as Series



In [25]:
%load_ext autoreload
%autoreload 2